In [31]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [163]:
def combine_features(row):
    #features = ['category', 'subcategory', 'prodname']
    return row['category'] + row['subcategory'] + row['prodname']

def get_name_index(index):
    return item_list[item_list.index == index]['prodname'].values

#print(get_name_index(0))
print(get_name_index(2304))

def get_index_name(prodname):
    return item_list[item_list.prodname == prodname].index.values[0]

def get_name_index_2(index):
    return item_list_int_F[item_list_int_F.index == index]['prodname'].values, item_list_int_F[item_list_int_F.index == index]['produnitprice'].values


def get_index_name_2(prodname):
    return item_list_int_F[item_list_int_F.prodname == prodname].index.values[0]

['Sustagen School 6+ 600g']


In [2]:
nba = pd.read_csv('malaysia_data/customer_receipt_product.csv', error_bad_lines=False)
now = pd.Timestamp('now')
nba['age'] = (now - pd.to_datetime(nba['dob'], format='%Y-%m-%d')).astype('<m8[Y]')

In [3]:
nba.head()

,customerid,receiptid,productid,category,subcategory,prodname,prodqty,produnitprice,produnitdiscount,productdetailsid,...,names,gender,race,dob,joineddate,emaildomain,city,postcode,state,age
0,5ae965218e3795fb61cbda69,1c3fd0dd-4cc2-4565-8be8-9bde1daeece5,1a5ae34b-cdbf-4827-b051-122eea978c32,Beverage,Pre-School 3yr onwards,Pediasure Complete 850g,1.0,158.16,NaN,49bcffa0-4822-4262-bc61-f658482020d9,...,Denise Mccarty,F,CHINESE,1990-07-20,2018-01-17,hotmail.com,Ipoh,31350.0,Perak,28.0
1,5ae965218e3795fb61cbda6a,8f226466-58fe-4cea-84af-939d653f7f45,47a327b5-f8a0-4b37-9688-4f709b1297f2,Personal Care,Anti-bacterial Wet Wipes,Antabax Cleansing Wipes 10s X 3pck,1.0,4.52,NaN,3cd77559-c3d3-4fe2-85c5-2a41e5aaad1c,...,Juan Norman,F,MALAY,1989-03-26,2018-03-09,gmail.com,Puchong,47100.0,Selangor,30.0
2,5ae965218e3795fb61cbda6b,155e7278-f7e8-4165-9b58-04fa2497b112,e4fa20e3-585f-4af0-b70d-33fceab0ee8f,Canned Food,Mackarel/ CuttleFish,Ayam Brand Black Beans Spicy Fried Mackerel 150g,1.0,5.45,NaN,7d4487e6-3b18-4241-a80d-342de867a0ab,...,Paula Gates,F,CHINESE,1982-09-20,2017-11-02,yahoo.com,NaN,NaN,Sabah,36.0
3,5ae965218e3795fb61cbda6b,2330c6cb-196d-4dc7-a191-07086708ff3e,d696ac67-c163-4306-baec-a3aa02ae56be,Milk Powder,Growing Up MIlk,Enfagrow A+ Step 4 Vanilla 600g,1.0,243.30,NaN,7e39d27e-d6f4-417e-830c-0a922f839bb8,...,Paula Gates,F,CHINESE,1982-09-20,2017-11-02,yahoo.com,NaN,NaN,Sabah,36.0
4,5ae965218e3795fb61cbda6b,155e7278-f7e8-4165-9b58-04fa2497b112,839d5ad5-244d-47ac-818e-adab309dd590,Household Products,Dishwashing Paste,Axion Dishwashing Paste 750g,1.0,5.65,NaN,44c0b518-9564-4b63-9ec3-88a3de85c7d4,...,Paula Gates,F,CHINESE,1982-09-20,2017-11-02,yahoo.com,NaN,NaN,Sabah,36.0


In [4]:
nba.columns

Index(['customerid', 'receiptid', 'productid', 'category', 'subcategory',
       'prodname', 'prodqty', 'produnitprice', 'produnitdiscount',
       'productdetailsid', 'submissionid', 'datetimeissued', 'receiptno',
       'retailer', 'retailertype', 'retailerarea', 'retailercity',
       'retailerstate', 'total', 'status', 'campaignid', 'retailerid',
       'clientid', 'membershipid', 'names', 'gender', 'race', 'dob',
       'joineddate', 'emaildomain', 'city', 'postcode', 'state', 'age'],
      dtype='object')

In [5]:
nba.count()
len(nba.prodname.unique())

3495

In [27]:
nba.groupby('prodname').count()[['category']].sort_values(by='category', ascending=False).head()

,category
prodname,
Natural Factors Calcium & Magnesium Plus C 90s,492
Kordel's Jflexi 30s,438
Kordel's Calcium + Vit D3 Plus Vit K2 60s,428
Kordel's Vitamin C 1000mg + Zinc Effervescent Tablet 10's PO4,419
Kordel's Vitamin C + Zinc Effervescent 10's,402


In [7]:
## Smaller list due to limited processing power. I.E. just choosing 100 items instead of 3495.

itemListSmall = nba.groupby('prodname').count()[['category']].sort_values(by='category', ascending=False).head(100)
itemListSmall = itemListSmall.index.values


In [8]:
itemList = list(set(nba['prodname'].tolist()))
userCount = len(set(nba['customerid'].tolist()))

### Calculate item affinity scores for items (Jaccard Similarity).

Similarity is based on the number of users which have rated item A and B divided by the number of users who have rated either A or B.

This is based on number of purchase for each item, Market Basket Analysis.

I.e. If I bought a laptop, I would have most likely bought carry case.

In [9]:
#Create an empty data frame to store item affinity scores for items.
itemAffinity= pd.DataFrame(columns=('item1', 'item2', 'score'))
rowCount=0
itemList = itemListSmall
#For each item in the list, compare with other items.
for ind1 in range(len(itemList)):
    
    #Get list of users who bought this item 1.
    item1Users = nba[nba.prodname==itemList[ind1]]["customerid"].tolist()
    #print("Item 1 ", item1Users)
    
    #Get item 2 - items that are not item 1 or those that are not analyzed already.
    for ind2 in range(ind1, len(itemList)):
        
        if ( ind1 == ind2):
            continue
       
        #Get list of users who bought item 2
        item2Users=nba[nba.prodname==itemList[ind2]]["customerid"].tolist()
        #print("Item 2",item2Users)
        
        #Find score. Find the common list of users and divide it by the total users.
        commonUsers= len(set(item1Users).intersection(set(item2Users)))
        score=commonUsers / userCount

        #Add a score for item 1, item 2
        itemAffinity.loc[rowCount] = [itemList[ind1],itemList[ind2],score]
        rowCount +=1
        #Add a score for item2, item 1. The same score would apply irrespective of the sequence.
        itemAffinity.loc[rowCount] = [itemList[ind2],itemList[ind1],score]
        rowCount +=1
        
#Check final result
itemAffinity.head()

,item1,item2,score
0,Natural Factors Calcium & Magnesium Plus C 90s,Kordel's Jflexi 30s,0.009761
1,Kordel's Jflexi 30s,Natural Factors Calcium & Magnesium Plus C 90s,0.009761
2,Natural Factors Calcium & Magnesium Plus C 90s,Kordel's Calcium + Vit D3 Plus Vit K2 60s,0.009219
3,Kordel's Calcium + Vit D3 Plus Vit K2 60s,Natural Factors Calcium & Magnesium Plus C 90s,0.009219
4,Natural Factors Calcium & Magnesium Plus C 90s,Kordel's Vitamin C 1000mg + Zinc Effervescent ...,0.011388


In [26]:
itemAffinity.sort_values(by='score', ascending=False).head()

,item1,item2,score,normScore
198,Kordel's Jflexi 30s,Kordel's Calcium + Vit D3 Plus Vit K2 60s,0.017354,0.99000
199,Kordel's Calcium + Vit D3 Plus Vit K2 60s,Kordel's Jflexi 30s,0.017354,0.99000
599,Kordel's Hyaluronic Acid With Collagen 90's + ...,Kordel's Vitamin C 1000mg + Zinc Effervescent ...,0.016811,0.96875
200,Kordel's Jflexi 30s,Kordel's Vitamin C 1000mg + Zinc Effervescent ...,0.016811,0.96875
608,Kordel's Vitamin C 1000mg + Zinc Effervescent ...,NuvaLanconone 50s,0.016811,0.96875


In [11]:
### Normalized score
itemAffinity['normScore'] = (itemAffinity['score'] - itemAffinity['score'].min()) / (itemAffinity['score'].max() - itemAffinity['score'].min())
itemAffinity.loc[itemAffinity.normScore == 1, 'normScore'] = 0.99

In [12]:
itemAffinity = itemAffinity.sort_values(by='score', ascending=False)

In [13]:
itemAffinity.sample(20)

,item1,item2,score,normScore
6388,Kordel's Garlic 3000mg 100s,Kordel's Kids Omega-3 Bites High DHA 50s,0.004881,0.28125
1767,The Prime Macuguard 60s+30s,Kordel's Hyaluronic Acid With Collagen 90's + ...,0.010846,0.62500
6389,Kordel's Kids Omega-3 Bites High DHA 50s,Kordel's Garlic 3000mg 100s,0.004881,0.28125
3208,Nuvafemme 60s+60s+20s,Kordel's C Time Acid Free C 1000mg 150's + 30's,0.009761,0.56250
6150,The Prime Macuguard 60s+30s,Kordel's Kids Omega-3 Bites High DHA 50s,0.007592,0.43750
838,Kordel's Vitamin C + Zinc Effervescent 10's,Natural Factors Krill Oil 500mg Sg 60 60s,0.005965,0.34375
6111,Kordel's Sea Mineral Complex 60's PO2,NuvaPine A 320mg (Double Strength) 100s+20s (KOL),0.006508,0.37500
9600,MSM Prai Coarse Sugar,Kordel's Bio Ester 1000mg 30's,0.000000,0.00000
6834,Kordel's V-Aid 30's,Kordel's Sea Mineral Complex 60's,0.006508,0.37500
8011,NuvaFlex B 2x80s,Natural Factors Milk Thistle Extract 60s x 2,0.007050,0.40625


In [15]:
# Getting the sample products

product1 = "Kordel's Vitamin C 1000mg + Zinc Effervescent Tablet 10's PO4"
product2 = "Natural Factors Milk Thistle Extract 60s x 2"
product3 = "Kordel's Hi-Omega Fish Oil 150's PO2"

In [16]:
sample1 = itemAffinity[itemAffinity['item1'] == product1]
sample2 = itemAffinity[itemAffinity['item1'] == product2]
sample3 = itemAffinity[itemAffinity['item1'] == product3]

In [17]:
array=[]
array.append(itemAffinity.iloc[5])
array.append(itemAffinity.iloc[len(itemAffinity) - int(len(itemAffinity)//1.05)])
array.append(itemAffinity.iloc[len(itemAffinity) - int(len(itemAffinity)//1.10)])
array.append(itemAffinity.iloc[len(itemAffinity) - int(len(itemAffinity)//1.15)])
array.append(itemAffinity.iloc[len(itemAffinity) - int(len(itemAffinity)//2)])
array.append(itemAffinity.iloc[len(itemAffinity) - int(len(itemAffinity)//3)])
array.append(itemAffinity.iloc[len(itemAffinity) - int(len(itemAffinity)//4)])
array.append(itemAffinity.iloc[len(itemAffinity) - int(len(itemAffinity)//5)])
array.append(itemAffinity.iloc[len(itemAffinity) - int(len(itemAffinity)//6)])
array.append(itemAffinity.iloc[len(itemAffinity) - int(len(itemAffinity)//7)])
array.append(itemAffinity.iloc[len(itemAffinity) - int(len(itemAffinity)//8)])

### Cosine Similarity Count Vectorizer (String categories, subcategories)

In [164]:
item_list = nba[['category', 'subcategory', 'prodname']].dropna().drop_duplicates().reset_index()
print(len(item_list))
item_list.head()

3542


,index,category,subcategory,prodname
0,0,Beverage,Pre-School 3yr onwards,Pediasure Complete 850g
1,1,Personal Care,Anti-bacterial Wet Wipes,Antabax Cleansing Wipes 10s X 3pck
2,2,Canned Food,Mackarel/ CuttleFish,Ayam Brand Black Beans Spicy Fried Mackerel 150g
3,3,Milk Powder,Growing Up MIlk,Enfagrow A+ Step 4 Vanilla 600g
4,4,Household Products,Dishwashing Paste,Axion Dishwashing Paste 750g


In [165]:
features = ['category', 'subcategory', 'prodname']

In [166]:
for feature in features:
    item_list[feature] = item_list[feature].fillna('')
    
item_list['combined_features'] = item_list.apply(combine_features, axis=1)

In [167]:
item_list['combined_features'].head(10)

0    BeveragePre-School 3yr onwardsPediasure Comple...
1    Personal CareAnti-bacterial Wet WipesAntabax C...
2    Canned FoodMackarel/ CuttleFishAyam Brand Blac...
3    Milk PowderGrowing Up MIlkEnfagrow A+ Step 4 V...
4    Household ProductsDishwashing PasteAxion Dishw...
5    Chilled & DairyCheeseArla Kids Cheese Sticks 108g
6    Chilled & DairyCheeseChesdale Cheddar Cheese 2...
7    Pasta & Instant NoodlesCup NoodlesMaggi Hot Cu...
8    Household ProductsConcentrated Fabric Conditio...
9          Baby DiapersDiaper PantsPetPet Daypants L66
Name: combined_features, dtype: object

In [168]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(item_list['combined_features'])

In [169]:
count_matrix

<3542x3575 sparse matrix of type '<class 'numpy.int64'>'
	with 25469 stored elements in Compressed Sparse Row format>

### Cosine Similarity
Similarity is the cosine of the angle between 2 vectors of the item vectors of A and B.

In [170]:
cosine_sim = cosine_similarity(count_matrix)

In [171]:
## Get values for sample
user_product = "Kordel's Garlic 3000mg 100s"
user_product_index = get_index_name(user_product)
#print(user_product_index)
similar_products = list(enumerate(cosine_sim[user_product_index]))

sorted_similar_product = sorted(similar_products,key=lambda x:x[1], reverse=True)[1:]

for i in range(10):
    element = sorted_similar_product[i]
    print(element[0])
    print(get_name_index(element[0]), element[1])

3322
["Kordel's Garlic 3000mg 500's + 100's"] 0.7715167498104596
3437
["Kordel's Echinacea & Garlic 50's"] 0.6666666666666669
3219
["Kordel's Jflexi 30s"] 0.5477225575051662
3225
["Kordel's Dichrotin 60's"] 0.5477225575051662
3328
["Kordel's V-Aid 30's"] 0.5477225575051662
3365
["Kordel's Executive B + C 50's"] 0.5477225575051662
3380
["Kordel's Executive B 60's"] 0.5477225575051662
3449
["Kordel's V-Aid 90's"] 0.5477225575051662
3467
["Kordel's Executive B + C 50sx2"] 0.5477225575051662
3476
["Kordel's Executive B 150's"] 0.5477225575051662


### Cosine similarity on integers

In [172]:
nba.head()

,customerid,receiptid,productid,category,subcategory,prodname,prodqty,produnitprice,produnitdiscount,productdetailsid,...,gender,race,dob,joineddate,emaildomain,city,postcode,state,age,combined_features
0,5ae965218e3795fb61cbda69,1c3fd0dd-4cc2-4565-8be8-9bde1daeece5,1a5ae34b-cdbf-4827-b051-122eea978c32,Beverage,Pre-School 3yr onwards,Pediasure Complete 850g,1.0,158.16,NaN,49bcffa0-4822-4262-bc61-f658482020d9,...,F,CHINESE,1990-07-20,2018-01-17,hotmail.com,Ipoh,31350.0,Perak,28.0,BeveragePre-School 3yr onwardsPediasure Comple...
1,5ae965218e3795fb61cbda6a,8f226466-58fe-4cea-84af-939d653f7f45,47a327b5-f8a0-4b37-9688-4f709b1297f2,Personal Care,Anti-bacterial Wet Wipes,Antabax Cleansing Wipes 10s X 3pck,1.0,4.52,NaN,3cd77559-c3d3-4fe2-85c5-2a41e5aaad1c,...,F,MALAY,1989-03-26,2018-03-09,gmail.com,Puchong,47100.0,Selangor,30.0,Personal CareAnti-bacterial Wet WipesAntabax C...
2,5ae965218e3795fb61cbda6b,155e7278-f7e8-4165-9b58-04fa2497b112,e4fa20e3-585f-4af0-b70d-33fceab0ee8f,Canned Food,Mackarel/ CuttleFish,Ayam Brand Black Beans Spicy Fried Mackerel 150g,1.0,5.45,NaN,7d4487e6-3b18-4241-a80d-342de867a0ab,...,F,CHINESE,1982-09-20,2017-11-02,yahoo.com,NaN,NaN,Sabah,36.0,Canned FoodMackarel/ CuttleFishAyam Brand Blac...
3,5ae965218e3795fb61cbda6b,2330c6cb-196d-4dc7-a191-07086708ff3e,d696ac67-c163-4306-baec-a3aa02ae56be,Milk Powder,Growing Up MIlk,Enfagrow A+ Step 4 Vanilla 600g,1.0,243.30,NaN,7e39d27e-d6f4-417e-830c-0a922f839bb8,...,F,CHINESE,1982-09-20,2017-11-02,yahoo.com,NaN,NaN,Sabah,36.0,Milk PowderGrowing Up MIlkEnfagrow A+ Step 4 V...
4,5ae965218e3795fb61cbda6b,155e7278-f7e8-4165-9b58-04fa2497b112,839d5ad5-244d-47ac-818e-adab309dd590,Household Products,Dishwashing Paste,Axion Dishwashing Paste 750g,1.0,5.65,NaN,44c0b518-9564-4b63-9ec3-88a3de85c7d4,...,F,CHINESE,1982-09-20,2017-11-02,yahoo.com,NaN,NaN,Sabah,36.0,Household ProductsDishwashing PasteAxion Dishw...


In [173]:
nba.columns

Index(['customerid', 'receiptid', 'productid', 'category', 'subcategory',
       'prodname', 'prodqty', 'produnitprice', 'produnitdiscount',
       'productdetailsid', 'submissionid', 'datetimeissued', 'receiptno',
       'retailer', 'retailertype', 'retailerarea', 'retailercity',
       'retailerstate', 'total', 'status', 'campaignid', 'retailerid',
       'clientid', 'membershipid', 'names', 'gender', 'race', 'dob',
       'joineddate', 'emaildomain', 'city', 'postcode', 'state', 'age',
       'combined_features'],
      dtype='object')

In [174]:
int_features = ['prodqty', 'produnitprice', 'produnitdiscount']

In [175]:
item_list_int_F = nba[['prodname', 'produnitprice', 'produnitdiscount', 'age']].drop_duplicates().fillna(0).reset_index()
print(len(item_list_int_F))

# Memory issue hence only taking the first 2000 items
item_list_int_F = item_list_int_F.head(2000)
item_list_int_F.head(100)

23708


,index,prodname,produnitprice,produnitdiscount,age
0,0,Pediasure Complete 850g,158.16,0.00,28.0
1,1,Antabax Cleansing Wipes 10s X 3pck,4.52,0.00,30.0
2,2,Ayam Brand Black Beans Spicy Fried Mackerel 150g,5.45,0.00,36.0
3,3,Enfagrow A+ Step 4 Vanilla 600g,243.30,0.00,36.0
4,4,Axion Dishwashing Paste 750g,5.65,0.00,36.0
5,5,Arla Kids Cheese Sticks 108g,12.50,0.00,34.0
6,6,Chesdale Cheddar Cheese 24 Slices 500g,21.60,0.00,34.0
7,7,Maggi Hot Cup 6 x 64g,9.35,1.40,34.0
8,8,Downy Parfum Collection Sweetheart Concentrate...,20.40,6.50,34.0
9,9,PetPet Daypants L66,26.90,0.00,34.0


Below is a bad example as it does not produnitdiscount is usually 0. It is only posisble for non 0 vectors.

In [179]:
from sklearn.metrics.pairwise import cosine_similarity

int_similarity = cosine_similarity(item_list_int_F[['produnitprice', 'produnitdiscount', 'age']])
#int_similarity = cosine_similarity(item_list_int_F[['produnitprice']])

In [180]:
## Get values for sample
user_product = "Pediasure Complete 850g"
user_product_index = get_index_name_2(user_product)
#print(user_product_index)
similar_products = list(enumerate(int_similarity[user_product_index]))

sorted_similar_product = sorted(similar_products,key=lambda x:x[1], reverse=True)[1:]

for i in range(10):
    element = sorted_similar_product[i]
    print(element[0])
    print(get_name_index_2(element[0]), element[1])

1
(array(['Antabax Cleansing Wipes 10s X 3pck'], dtype=object), array([4.52])) 1.0
2
(array(['Ayam Brand Black Beans Spicy Fried Mackerel 150g'], dtype=object), array([5.45])) 1.0
3
(array(['Enfagrow A+ Step 4 Vanilla 600g'], dtype=object), array([243.3])) 1.0
4
(array(['Axion Dishwashing Paste 750g'], dtype=object), array([5.65])) 1.0
5
(array(['Arla Kids Cheese Sticks 108g'], dtype=object), array([12.5])) 1.0
6
(array(['Chesdale Cheddar Cheese 24 Slices 500g'], dtype=object), array([21.6])) 1.0
7
(array(['Maggi Hot Cup 6 x 64g'], dtype=object), array([9.35])) 1.0
8
(array(['Downy Parfum Collection Sweetheart Concentrate Fabric Conditioner Refill 1.5L'],
      dtype=object), array([20.4])) 1.0
9
(array(['PetPet Daypants L66'], dtype=object), array([26.9])) 1.0
10
(array(['Nestle Cheerios 300g'], dtype=object), array([10.85])) 1.0


### Below are for output and testing purposes. Esp during the datathon

In [22]:
sample1 = sample1.sort_values(by='score', ascending=False)
array1=[]
#array1.append(sample1.iloc[5])
array1.append(sample1.iloc[len(sample1) - int(len(sample1)//1.05)])
array1.append(sample1.iloc[len(sample1) - int(len(sample1)//1.10)])
array1.append(sample1.iloc[len(sample1) - int(len(sample1)//1.15)])
array1.append(sample1.iloc[len(sample1) - int(len(sample1)//2)])
array1.append(sample1.iloc[len(sample1) - int(len(sample1)//3)])
array1.append(sample1.iloc[len(sample1) - int(len(sample1)//4)])
array1.append(sample1.iloc[len(sample1) - int(len(sample1)//5)])
array1.append(sample1.iloc[len(sample1) - int(len(sample1)//6)])
array1.append(sample1.iloc[len(sample1) - int(len(sample1)//7)])
array1.append(sample1.iloc[len(sample1) - int(len(sample1)//8)])

#for element in array1:
    #print(element)

In [23]:
sample2 = sample2.sort_values(by='score', ascending=False)
array2=[]
#array2.append(sample2.iloc[5])
array2.append(sample2.iloc[len(sample2) - int(len(sample2)//1.05)])
array2.append(sample2.iloc[len(sample2) - int(len(sample2)//1.10)])
array2.append(sample2.iloc[len(sample2) - int(len(sample2)//1.15)])
array2.append(sample2.iloc[len(sample2) - int(len(sample2)//2)])
array2.append(sample2.iloc[len(sample2) - int(len(sample2)//3)])
array2.append(sample2.iloc[len(sample2) - int(len(sample2)//4)])
array2.append(sample2.iloc[len(sample2) - int(len(sample2)//5)])
array2.append(sample2.iloc[len(sample2) - int(len(sample2)//6)])
array2.append(sample2.iloc[len(sample2) - int(len(sample2)//7)])
array2.append(sample2.iloc[len(sample2) - int(len(sample2)//8)])

#for element in array2:
    #print(element)

In [24]:
sample3 = sample3.sort_values(by='score', ascending=False)
array3=[]
#array3.append(sample3.iloc[5])
array3.append(sample3.iloc[len(sample3) - int(len(sample3)//1.05)])
array3.append(sample3.iloc[len(sample3) - int(len(sample3)//1.10)])
array3.append(sample3.iloc[len(sample3) - int(len(sample3)//1.15)])
array3.append(sample3.iloc[len(sample3) - int(len(sample3)//2)])
array3.append(sample3.iloc[len(sample3) - int(len(sample3)//3)])
array3.append(sample3.iloc[len(sample3) - int(len(sample3)//4)])
array3.append(sample3.iloc[len(sample3) - int(len(sample3)//5)])
array3.append(sample3.iloc[len(sample3) - int(len(sample3)//6)])
array3.append(sample3.iloc[len(sample3) - int(len(sample3)//7)])
array3.append(sample3.iloc[len(sample3) - int(len(sample3)//8)])

#for element in array3:
    #print(element)

In [25]:
for element in array:
    print(element[0])

Kordel's Vitamin C 1000mg + Zinc Effervescent Tablet 10's PO4
Kordel's Calcium + Vit D3 Plus Vit K2 60s
Kordel's Eggshell Membrane + Boswellia 90's + 30's
NuvaLanconone 50s
Kordel's Vitamin C + Zinc Effervescent 10's
Kordel's Eggshell Membrane + Boswellia 30's
Kordel's Gluthathione ALA Plus 60s
Kordel's Nutri-Time Multi Vitamins and Minerals 90's + 30's
Kordel's Bio Ester 1000mg 30's
Kordel's Omega 3 Fish Oil 120's PO2
Safwa Health Glucosamine Plus Chondroitin PO2 2x50


In [66]:
len(array)

2

In [105]:
### Save Results to array
import psycopg2
connection = psycopg2.connect(user='o1',
                              password='601111444097',
                              host='hackaton-db.fusionexgiant.com',
                              port='35432',
                              database='o1')

postgres_insert_query = """INSERT INTO nba1 (item1, item2, score, norm) VALUES (%s, %s, %s, %s)"""
cursor = connection.cursor()
for element in array1:
    cursor.execute(postgres_insert_query, element)
    connection.commit()


In [106]:
### Save Results to array
import psycopg2
connection = psycopg2.connect(user='o1',
                              password='601111444097',
                              host='hackaton-db.fusionexgiant.com',
                              port='35432',
                              database='o1')

postgres_insert_query = """INSERT INTO nba2 (item1, item2, score, norm) VALUES (%s, %s, %s, %s)"""
cursor = connection.cursor()
for element in array2:
    cursor.execute(postgres_insert_query, element)
    connection.commit()


In [107]:
### Save Results to array
import psycopg2
connection = psycopg2.connect(user='o1',
                              password='601111444097',
                              host='hackaton-db.fusionexgiant.com',
                              port='35432',
                              database='o1')

postgres_insert_query = """INSERT INTO nba3 (item1, item2, score, norm) VALUES (%s, %s, %s, %s)"""
cursor = connection.cursor()
for element in array3:
    cursor.execute(postgres_insert_query, element)
    connection.commit()


In [139]:
customer_list = nba['customerid'].sample(20)

In [140]:
customer_list = customer_list.to_list()

In [141]:
for element in customer_list:
    print(str(element))

5b692da88e3795fb61a9136f
5b6c00238e3795fb61d04062
5b6833ed8e3795fb619b8729
5b6a91d28e3795fb61bc57dd
5b6828ef8e3795fb619aeae3
5b3c98848e3795fb61385946
5b1e35f98e3795fb611de20a
5b7b8cf68e3795fb61a8254c
5b76820a8e3795fb61620ef3
5b7b8cf68e3795fb61a8254c
5b6afbba8e3795fb61c22919
5b692da88e3795fb61a9136f
5b692da88e3795fb61a9136f
5b2f8a868e3795fb6192bd18
5b7ed7f78e3795fb61d6c944
5b080e5a8e3795fb61cea06c
5b7ed7f78e3795fb61d6c944
5b6825578e3795fb619ab8a3
5b2b58c48e3795fb6168dee5
5b6930e58e3795fb61a94267


In [142]:
### Save Results to array
import psycopg2
connection = psycopg2.connect(user='o1',
                              password='601111444097',
                              host='hackaton-db.fusionexgiant.com',
                              port='35432',
                              database='o1')

postgres_insert_query = """INSERT INTO item3_marketing_list (customer_id) VALUES (%s)"""
cursor = connection.cursor()
for element in customer_list:
    cursor.execute(postgres_insert_query, [element])
    connection.commit()

In [ ]:
nba.groupby('prodname').count()[['category']].sort_values(by='category', ascending=False)

In [143]:
nba.columns

Index(['customerid', 'receiptid', 'productid', 'category', 'subcategory',
       'prodname', 'prodqty', 'produnitprice', 'produnitdiscount',
       'productdetailsid', 'submissionid', 'datetimeissued', 'receiptno',
       'retailer', 'retailertype', 'retailerarea', 'retailercity',
       'retailerstate', 'total', 'status', 'campaignid', 'retailerid',
       'clientid', 'membershipid', 'names', 'gender', 'race', 'dob',
       'joineddate', 'emaildomain', 'city', 'postcode', 'state'],
      dtype='object')

In [222]:
nba_sample1 = nba[nba['prodname'] == product1]
nba_sample1.head()
nba_sample1.customerid.unique()
nba_sample1.columns

Index(['customerid', 'receiptid', 'productid', 'category', 'subcategory',
       'prodname', 'prodqty', 'produnitprice', 'produnitdiscount',
       'productdetailsid', 'submissionid', 'datetimeissued', 'receiptno',
       'retailer', 'retailertype', 'retailerarea', 'retailercity',
       'retailerstate', 'total', 'status', 'campaignid', 'retailerid',
       'clientid', 'membershipid', 'names', 'gender', 'race', 'dob',
       'joineddate', 'emaildomain', 'city', 'postcode', 'state', 'age'],
      dtype='object')

In [230]:
test2 = nba_sample1.groupby(['gender', 'race']).mean()['age']

In [231]:
test2

gender  race   
F       CHINESE    47.102210
        INDIAN     46.000000
        MALAY      47.178571
Name: age, dtype: float64

In [224]:
test2 = nba_sample1.groupby(['gender', 'race']).count()['receiptid']

In [225]:
test2

gender  race   
F       CHINESE    362
        INDIAN       1
        MALAY       56
Name: receiptid, dtype: int64

In [191]:
test2 = pd.read_csv('test2.csv')

In [188]:
test2.head()

,Unnamed: 0,0,1
0,0,5b6825578e3795fb619ab8a3,customerid ...
1,1,5b6828ef8e3795fb619aeae3,customerid ...
2,2,5b682c508e3795fb619b1a6a,customerid ...
3,3,5b6833ed8e3795fb619b8729,customerid ...
4,4,5b6835fe8e3795fb619ba4dd,customerid ...


In [235]:

nba.columns
nba['datetimeissued'].head()

0    2018-08-30 07:01:00
1    2018-07-03 04:27:00
2    2018-08-03 10:40:00
3    2018-09-01 08:03:00
4    2018-08-03 10:40:00
Name: datetimeissued, dtype: object

In [243]:
nba['purchased_timeframe'] = (now - pd.to_datetime(nba['datetimeissued'], format='%Y-%m-%d'))

In [244]:
nba['purchased_timeframe'].head()

0   317 days 17:45:01.171536
1   375 days 20:19:01.171536
2   344 days 14:06:01.171536
3   315 days 16:43:01.171536
4   344 days 14:06:01.171536
Name: purchased_timeframe, dtype: timedelta64[ns]

In [245]:
nba.sort_values(by=['customerid', 'purchased_timeframe'])

,customerid,receiptid,productid,category,subcategory,prodname,prodqty,produnitprice,produnitdiscount,productdetailsid,...,gender,race,dob,joineddate,emaildomain,city,postcode,state,age,purchased_timeframe
0,5ae965218e3795fb61cbda69,1c3fd0dd-4cc2-4565-8be8-9bde1daeece5,1a5ae34b-cdbf-4827-b051-122eea978c32,Beverage,Pre-School 3yr onwards,Pediasure Complete 850g,1.0,158.16,NaN,49bcffa0-4822-4262-bc61-f658482020d9,...,F,CHINESE,1990-07-20,2018-01-17,hotmail.com,Ipoh,31350.0,Perak,28.0,317 days 17:45:01.171536
1,5ae965218e3795fb61cbda6a,8f226466-58fe-4cea-84af-939d653f7f45,47a327b5-f8a0-4b37-9688-4f709b1297f2,Personal Care,Anti-bacterial Wet Wipes,Antabax Cleansing Wipes 10s X 3pck,1.0,4.52,NaN,3cd77559-c3d3-4fe2-85c5-2a41e5aaad1c,...,F,MALAY,1989-03-26,2018-03-09,gmail.com,Puchong,47100.0,Selangor,30.0,375 days 20:19:01.171536
3,5ae965218e3795fb61cbda6b,2330c6cb-196d-4dc7-a191-07086708ff3e,d696ac67-c163-4306-baec-a3aa02ae56be,Milk Powder,Growing Up MIlk,Enfagrow A+ Step 4 Vanilla 600g,1.0,243.30,NaN,7e39d27e-d6f4-417e-830c-0a922f839bb8,...,F,CHINESE,1982-09-20,2017-11-02,yahoo.com,NaN,NaN,Sabah,36.0,315 days 16:43:01.171536
2,5ae965218e3795fb61cbda6b,155e7278-f7e8-4165-9b58-04fa2497b112,e4fa20e3-585f-4af0-b70d-33fceab0ee8f,Canned Food,Mackarel/ CuttleFish,Ayam Brand Black Beans Spicy Fried Mackerel 150g,1.0,5.45,NaN,7d4487e6-3b18-4241-a80d-342de867a0ab,...,F,CHINESE,1982-09-20,2017-11-02,yahoo.com,NaN,NaN,Sabah,36.0,344 days 14:06:01.171536
4,5ae965218e3795fb61cbda6b,155e7278-f7e8-4165-9b58-04fa2497b112,839d5ad5-244d-47ac-818e-adab309dd590,Household Products,Dishwashing Paste,Axion Dishwashing Paste 750g,1.0,5.65,NaN,44c0b518-9564-4b63-9ec3-88a3de85c7d4,...,F,CHINESE,1982-09-20,2017-11-02,yahoo.com,NaN,NaN,Sabah,36.0,344 days 14:06:01.171536
9,5aec2e818e3795fb6113cbed,bc8185a8-3c66-47b7-bec1-37f545ebc48a,2ddf96f0-bddc-474c-bf93-8566170cc701,Baby Diapers,Diaper Pants,PetPet Daypants L66,3.0,26.90,NaN,6a0d5576-c8a1-4552-9b09-77340d52319b,...,F,CHINESE,1985-01-06,2017-04-08,hotmail.com,Petaling Jaya,47301.0,Selangor,34.0,342 days 12:41:01.171536
20,5aec2e818e3795fb6113cbed,bc8185a8-3c66-47b7-bec1-37f545ebc48a,7472c413-4bb7-4082-95bf-d7986d43f931,Beverage,Pasteurised Milk,Farmhouse Fresh Pasteurised Milk 1L x 2,2.0,18.39,NaN,64f36712-16cd-4370-afc2-46dc2af6c568,...,F,CHINESE,1985-01-06,2017-04-08,hotmail.com,Petaling Jaya,47301.0,Selangor,34.0,342 days 12:41:01.171536
5,5aec2e818e3795fb6113cbed,65688e1a-c40a-475d-a79b-04046c63a3af,bddb0fd3-3608-4860-8011-fc8c3a1f3dc8,Chilled & Dairy,Cheese,Arla Kids Cheese Sticks 108g,1.0,12.50,NaN,5134e8d6-9d7b-4ad6-814b-42d941f68505,...,F,CHINESE,1985-01-06,2017-04-08,hotmail.com,Petaling Jaya,47301.0,Selangor,34.0,468 days 16:02:01.171536
6,5aec2e818e3795fb6113cbed,65688e1a-c40a-475d-a79b-04046c63a3af,2cdb3482-03eb-44f6-9f95-4ac845e4e57a,Chilled & Dairy,Cheese,Chesdale Cheddar Cheese 24 Slices 500g,1.0,21.60,NaN,a62bbb32-cf01-44d4-9f45-297adfae8845,...,F,CHINESE,1985-01-06,2017-04-08,hotmail.com,Petaling Jaya,47301.0,Selangor,34.0,468 days 16:02:01.171536
7,5aec2e818e3795fb6113cbed,65688e1a-c40a-475d-a79b-04046c63a3af,b63bca35-0037-417c-b458-489e75de3c7c,Pasta & Instant Noodles,Cup Noodles,Maggi Hot Cup 6 x 64g,1.0,9.35,1.4,9f62209d-e98c-4ae5-88a9-261551e9d27a,...,F,CHINESE,1985-01-06,2017-04-08,hotmail.com,Petaling Jaya,47301.0,Selangor,34.0,468 days 16:02:01.171536


In [249]:
churn = nba.groupby('customerid').agg({'purchased_timeframe': 'min'}).sort_values(by='purchased_timeframe')

In [251]:
churn['pred'] = (churn['purchased_timeframe'] - churn['purchased_timeframe'].min())/(churn['purchased_timeframe'].max()-churn['purchased_timeframe'].min())

In [252]:
churn.head()

,purchased_timeframe,pred
customerid,,
5b72be378e3795fb612dbbe2,173 days 22:33:01.171536,0.000000
5b6922e88e3795fb61a87833,197 days 16:46:01.171536,0.071590
5b6828ef8e3795fb619aeae3,197 days 17:20:01.171536,0.071661
5b6afbba8e3795fb61c22919,197 days 17:20:01.171536,0.071661
5b7a706b8e3795fb6198927d,197 days 19:20:01.171536,0.071912


In [264]:
churn.tail()
churn['id'] = churn.index.values
churn_result = churn.values.tolist()

In [265]:
churn_result

[[Timedelta('173 days 22:33:01.171536'), 0.0, '5b72be378e3795fb612dbbe2'],
 [Timedelta('197 days 16:46:01.171536'),
  0.07158954185455269,
  '5b6922e88e3795fb61a87833'],
 [Timedelta('197 days 17:20:01.171536'),
  0.07166068570113307,
  '5b6828ef8e3795fb619aeae3'],
 [Timedelta('197 days 17:20:01.171536'),
  0.07166068570113307,
  '5b6afbba8e3795fb61c22919'],
 [Timedelta('197 days 19:20:01.171536'),
  0.07191178163024033,
  '5b7a706b8e3795fb6198927d'],
 [Timedelta('197 days 21:27:01.171536'),
  0.07217752482187882,
  '5b682c508e3795fb619b1a6a'],
 [Timedelta('198 days 15:16:01.171536'),
  0.07441437105700924,
  '5b6970a88e3795fb61acb8c2'],
 [Timedelta('198 days 15:31:01.171536'),
  0.07444575804814764,
  '5b7ed7f78e3795fb61d6c944'],
 [Timedelta('198 days 17:04:01.171536'),
  0.07464035739320576,
  '5b696d648e3795fb61ac8b1c'],
 [Timedelta('198 days 17:59:01.171536'),
  0.07475544302737992,
  '5b6c00238e3795fb61d04062'],
 [Timedelta('198 days 18:06:01.171536'),
  0.07477009028991118,
  '5b6

In [268]:
### Save Results to array
import psycopg2
connection = psycopg2.connect(user='o1',
                              password='601111444097',
                              host='hackaton-db.fusionexgiant.com',
                              port='35432',
                              database='o1')

postgres_insert_query = """INSERT INTO churn_list (purchased_timeframe, pred_score, customer_id) VALUES (%s, %s, %s)"""
cursor = connection.cursor()
for element in churn_result:
    cursor.execute(postgres_insert_query, element)
    connection.commit()

In [267]:
import psycopg2

try:
    conn = psycopg2.connect(database = "o1", user = "o1", password = "601111444097", host = "hackaton-db.fusionexgiant.com", port = "35432")
except:
    print("I am unable to connect to the database") 

cur = conn.cursor()
cur.execute("""CREATE TABLE churn_list(
                
                purchased_timeframe VARCHAR,
                pred_score float,
                customer_id VARCHAR
            );""")



conn.commit() # <--- makes sure the change is shown in the database
conn.close()
cur.close()